High Value Customer Identification (Insiders)

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import re

In [3]:
text00 = '234.745.735-42'

regex = '^\d{3}\.\d{3}\.\d{3}\-\d{2}$'

In [4]:
bool( re.match(regex, text00) )

True

In [5]:
data = pd.read_csv("df_products.csv")
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
0,690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,690449043,Only non-chlorine bleach when needed,NaN,"Cotton 98%, Spandex 2%",DENIM,Light denim blue,Skinny fit,Yes,keyFibreType,NaN,690449,43
1,690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,690449043,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Light denim blue,Skinny fit,Yes,Denim,NaN,690449,43
2,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-29 16:18:45,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Slim fit,Yes,Denim,NaN,1024256,1
3,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-29 16:18:45,NaN,1024256001,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Solid-color,Slim fit,Yes,Denim,NaN,1024256,1
4,985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-29 16:18:45,Recycled cotton 20%,985159001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Skinny fit,Yes,Denim,NaN,985159,1


In [10]:
data.isnull().sum()

product_id                          0
product_category                    0
product_name                        0
product_price                       0
scrapy_datetime                     0
additional material information    52
art. no.                            0
care instructions                   0
collection                         65
composition                         0
concept                             6
description                         0
fit                                 0
imported                            0
material                            0
size                               59
style_id                            0
color_id                            0
dtype: int64

In [21]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
0,690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,690449043,Only non-chlorine bleach when needed,NaN,"Cotton 98%, Spandex 2%",DENIM,Light denim blue,Skinny fit,Yes,keyFibreType,NaN,690449,43
1,690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,690449043,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Light denim blue,Skinny fit,Yes,Denim,NaN,690449,43
2,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-29 16:18:45,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Slim fit,Yes,Denim,NaN,1024256,1
3,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-29 16:18:45,NaN,1024256001,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Solid-color,Slim fit,Yes,Denim,NaN,1024256,1
4,985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-29 16:18:45,Recycled cotton 20%,985159001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Skinny fit,Yes,Denim,NaN,985159,1


In [22]:
data2 = data.copy()

for i in ['style_id', 'color_id', 'product_id']:
    data2[i] = data2[i].astype(int)

for i in ['product_name', 'fit', 'description', 'material']:
    data2[i] = data2[i].apply( lambda x: x.replace(" ", "_").lower())

data2['product_price'] = data2['product_price'].apply( lambda x: x.replace("$", "")).astype(float)

data2['scrapy_datetime'] = pd.to_datetime(data2['scrapy_datetime'], format = "%Y-%m-%d %H:%M:%S")




data2.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
0,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-11-29 16:18:45,NaN,690449043,Only non-chlorine bleach when needed,NaN,"Cotton 98%, Spandex 2%",DENIM,light_denim_blue,skinny_fit,Yes,keyfibretype,NaN,690449,43
1,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-11-29 16:18:45,NaN,690449043,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,light_denim_blue,skinny_fit,Yes,denim,NaN,690449,43
2,1024256001,men_jeans_slim,slim_jeans,19.99,2022-11-29 16:18:45,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,black,slim_fit,Yes,denim,NaN,1024256,1
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-11-29 16:18:45,NaN,1024256001,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,solid-color,slim_fit,Yes,denim,NaN,1024256,1
4,985159001,men_jeans_skinny,skinny_jeans,24.99,2022-11-29 16:18:45,Recycled cotton 20%,985159001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,black,skinny_fit,Yes,denim,NaN,985159,1
